In [1]:
PIANO_GENERATOR_MODEL_PATH = 'piano_model'
PIANO_GENERATOR_MODEL_WEIGHTS_PATH = './piano models/piano_model_autoencoder_weights'

In [5]:
import os

import glob
import pickle
import numpy
import pandas
import numpy as np
from itertools import chain
from pydub import AudioSegment
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import keras
from keras import layers

In [6]:
def get_notes(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [7]:
def get_notes_v2(path):
    """ Get all the notes and chords from the midi files in the 'path' directory """
    notes = []

    print(len(glob.glob(path + '/*.mid')))
    for file in glob.glob(path + '/*.mid'):
        midi = converter.parse(file)

#         print('parsing %s' % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        
        file_notes = []
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                file_notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                file_notes.append('.'.join(str(n) for n in element.normalOrder))
        
        notes.append(file_notes)
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [5]:
notes = get_notes_v2('midi_songs')

93


In [6]:
len(notes)

93

In [7]:
notes_len = [len(file_notes) for file_notes in notes]

In [8]:
np.mean(notes_len), min(notes_len)

(20.43010752688172, 0)

In [9]:
sorted(notes_len)[:10]

[0, 3, 4, 4, 5, 5, 5, 6, 6, 6]

In [10]:
sum(np.array(notes_len) >= 8)

79

In [6]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [14]:
print(len(notes))
len([note for note in notes if len(note) >= 8])

93


79

In [ ]:
def prepare_sequences_v2(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [18]:
notes

[['2.3.7.10',
  '10.2',
  '7.10.0.3',
  '10.2',
  '5.9',
  '10.2',
  '3.7',
  'D4',
  '3.7',
  '10.2',
  '0.3.7',
  '10',
  '10.2',
  '5.9',
  '10.2'],
 ['6.9.11.2',
  'D5',
  '4.7.9.0',
  '11.0.4.7',
  'F#5',
  'E5',
  'D5',
  'B4',
  '6.9.11.2',
  'D5',
  '0.4.7',
  '11.0.4.7',
  '6.7.11.2'],
 ['8.11', '3.6', '8.11', 'E-4', 'B3', '8.11.3', 'F#4', '8.11.3'],
 ['4.6.9.11',
  '1',
  '11',
  '9.11.1.4',
  'C#4',
  '9.11.1.4',
  '8.11.1.4',
  '1',
  '7.10.0.3',
  'C#4',
  'B1',
  '2.6.9',
  '4',
  'C#4',
  '11',
  'B1',
  '2.6.9',
  'C#4',
  'D2',
  '6.9.1',
  'C#4',
  'C#4',
  '9',
  '4.6.9.11',
  '1',
  '11',
  '9.11.1.4',
  'C#4',
  '9.11.1.4',
  '8.11.1.4',
  '1',
  '7.10.0.3',
  'C#4',
  'B1',
  '2.6.9',
  '4',
  'C#4',
  '6',
  'B1',
  '2.6.9',
  'C#4',
  'D2',
  '6.9.1',
  '1.4',
  '9.1'],
 ['11.4',
  '7',
  'F#4',
  'G3',
  'D5',
  'B5',
  '11.0',
  '9.2',
  '7.11.2',
  'B5',
  'F#4',
  '7.11.2',
  '7.0',
  '9.0',
  '0.4.7',
  '7',
  '7.0',
  'F#4',
  '11.2',
  'B5',
  '7.0',
  '0

In [17]:
for i in range(0, 1, 1):
    print(i)

0


In [57]:
def prepare_sequences_v2(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 8

    notes = [note for note in notes if len(note) >= sequence_length]
    
    # get all pitch names
    pitchnames = sorted(set(item for item in list(chain.from_iterable(notes))))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for file_note in notes:
        for i in range(0, len(file_note) - sequence_length + 1, 1):
            sequence_in = file_note[i:i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    return (network_input, network_output)

In [58]:
n_vocab = len(set(list(chain.from_iterable(notes))))
network_input, network_output = prepare_sequences_v2(notes, n_vocab)

In [59]:
len(notes)

93

In [60]:
len(network_input), len(network_input[0])

(1276, 8)

In [44]:
network_input[0]

array([[0.25590551],
       [0.17322835],
       [0.58267717],
       [0.17322835],
       [0.48818898],
       [0.17322835],
       [0.37795276],
       [0.85433071]])

In [65]:
def create_autoencoder_network():
    """ create the structure of the neural network """

    # This is the size of our encoded representations
    encoding_dim = 4

    # This is our input image
    input_img = keras.Input(shape=(8,))
    # "encoded" is the encoded representation of the input
    encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
    # "decoded" is the lossy reconstruction of the input
    decoded = layers.Dense(8, activation='sigmoid')(encoded)

    # This model maps an input to its reconstruction
    autoencoder = keras.Model(input_img, decoded)
    
    return autoencoder

In [72]:
encoder = create_autoencoder_network()

In [73]:
encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [74]:
encoder.fit(network_input, network_input,
            epochs=100,
            batch_size=32,
            shuffle=True)

Epoch 1/100
40/40 [==============================] - 0s 914us/step - loss: 0.6976
Epoch 2/100
40/40 [==============================] - 0s 634us/step - loss: 0.6821
Epoch 3/100
40/40 [==============================] - 0s 544us/step - loss: 0.6713
Epoch 4/100
40/40 [==============================] - 0s 547us/step - loss: 0.6615
Epoch 5/100
40/40 [==============================] - 0s 528us/step - loss: 0.6521
Epoch 6/100
40/40 [==============================] - 0s 522us/step - loss: 0.6429
Epoch 7/100
40/40 [==============================] - 0s 524us/step - loss: 0.6334
Epoch 8/100
40/40 [==============================] - 0s 553us/step - loss: 0.6240
Epoch 9/100
40/40 [==============================] - 0s 549us/step - loss: 0.6155
Epoch 10/100
40/40 [==============================] - 0s 577us/step - loss: 0.6086
Epoch 11/100
40/40 [==============================] - 0s 542us/step - loss: 0.6034
Epoch 12/100
40/40 [==============================] - 0s 535us/step - loss: 0.5994
Epoch 13/100


In [ ]:
note

In [12]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "updated-lofi-hip-hop-weights-improvement-1{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

In [13]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [38]:
model = create_network(network_input, n_vocab)

In [39]:
model.fit(network_input, network_output, epochs=30, batch_size=64, callbacks=callbacks_list)

Epoch 1/30
30/30 [==============================] - 32s 958ms/step - loss: 5.9099
Epoch 2/30
30/30 [==============================] - 28s 921ms/step - loss: 5.6505
Epoch 3/30
30/30 [==============================] - 31s 1s/step - loss: 5.5291
Epoch 4/30
30/30 [==============================] - 38s 1s/step - loss: 5.5055
Epoch 5/30
30/30 [==============================] - 30s 1s/step - loss: 5.3513
Epoch 6/30
30/30 [==============================] - 31s 1s/step - loss: 5.3021
Epoch 7/30
30/30 [==============================] - 32s 1s/step - loss: 5.2602
Epoch 8/30
30/30 [==============================] - 31s 1s/step - loss: 5.1564
Epoch 9/30
30/30 [==============================] - 31s 1s/step - loss: 5.1468
Epoch 10/30
30/30 [==============================] - 31s 1s/step - loss: 5.0318
Epoch 11/30
30/30 [==============================] - 31s 1s/step - loss: 5.0592
Epoch 12/30
30/30 [==============================] - 32s 1s/step - loss: 5.0199
Epoch 13/30
30/30 [========================

In [18]:
model.save(PIANO_GENERATOR_MODEL_PATH)

INFO:tensorflow:Assets written to: piano_model/assets


INFO:tensorflow:Assets written to: piano_model/assets


In [40]:
model.save_weights(PIANO_GENERATOR_MODEL_WEIGHTS_PATH + '_02')

## generate piano roll midi file

In [25]:
def load_learned_network(network_input, n_vocab, weights_path):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Load the weights to each node
    model.load_weights(weights_path)

    return model

In [26]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(200):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [27]:
def create_midi(prediction_output, 
                output_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
#         offset += 0.5
        offset += 1

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=output_path)

In [28]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 32
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [32]:
def generate(weights_path, output_path):
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = load_learned_network(normalized_input, n_vocab, weights_path)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output, output_path)

In [46]:
generate(weights_path=PIANO_GENERATOR_MODEL_WEIGHTS_PATH+'_01', output_path='lofi_output7.mid')

In [58]:
_ = os.system('fluidsynth -ni Touhou.sf2 lofi_output7.mid -F piano_generated_07.wav -r 44100')